In [9]:
import os
import mlflow
import pandas as pd
import plotly.express as px
import sklearn.pipeline
from mlflow.tracking import MlflowClient

In [49]:
def get_features(model_name: str, stage: str):
    client = mlflow.tracking.MlflowClient()
    model_version = client.get_latest_versions(name=model_name, stages=[stage])[0]
    run_id = model_version.run_id
    run = client.get_run(run_id)
    features = run.data.params["features"]
    return features.split(", ")

In [51]:
mlflow.set_tracking_uri("http://20.31.225.193")


In [66]:
model_name = "best_model"
stage = "production"

#load run using model_name and stage
model = mlflow.sklearn.load_model(f"models:/{model_name}/{stage}")

model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('randomforestregressor', RandomForestRegressor())])

In [65]:
features = get_features(model_name, stage)
features

["Index(['wind_speed'", "'wind_direction']", "dtype='object')"]

In [59]:
# Load data
data_url = "https://raw.githubusercontent.com/ykerus/experiment-tracking-with-mlflow/main/data/scada.parquet" # noqa
data = pd.read_parquet(data_url)

In [60]:
# Inference
data_without_na = data.dropna()
X = data_without_na[features]
y = data_without_na["active_power"]

data.loc[X.index, "predictions"] = model.predict(X)

In [61]:
score = model.score(X, y)

In [62]:
score

0.7691263879901377

In [63]:
# Plot
plot_start = pd.Timestamp("2018-12-15")
fig = px.line(
    data[plot_start:],
    y=["active_power", "predictions"]
)
fig